In [1]:
import json
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
#import random
import numpy as np
import pickle
import tflearn
import tensorflow.compat.v1 as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense, Input, Dropout, Activation 
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

Instructions for updating:
non-resource variables are not supported in the long term


## Language understanding 

### Load data

In [2]:
# Reading the data file and axetracting the training data
f = open("intents.json")
file = json.loads(f.read())

text = []
label = []

for item in file['intents']:
    for text_id in item['patterns']:
        text.append(text_id) 
        label.append(item['tag'])
    #if file['tag'] not in label:
          #  label.append(file['tag'])

print(text)
print(label)

['Hi there', 'How are you', 'Is anyone there?', 'Hello', 'Good day', 'Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time', 'Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me', 'How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered', 'How to check Adverse drug reaction?', 'Open adverse drugs module', 'Give me a list of drugs causing adverse behavior', 'List all drugs suitable for patient with adverse reaction', 'Which drugs dont have adverse reaction?', 'Open blood pressure module', 'Task related to blood pressure', 'Blood pressure data entry', 'I want to log blood pressure results', 'Blood pressure data management', 'I want to search for blood pressure result history', 'Blood pressure for patient', 'Load patient blood pressure result', 'Show blood pressure results for patient', 'Find blood pressure results by ID', 'Find me a pharmacy', 'Find pharmacy', 'List 

### Data Pre-processing 

In [3]:
#dividing the data into training and testintg data by a ratio of 80:20
'''size = int(len(text)*0.8)
split_text = [text[i:i + size] for i in range(0, len(text), size)]
split_label = [label[i:i + size] for i in range(0, len(text), size)]

train_text_data = split_text[0]
test_text_data = split_text[1]
train_label_data = split_label[0]
test_label_data = split_label[1]'''

'size = int(len(text)*0.8)\nsplit_text = [text[i:i + size] for i in range(0, len(text), size)]\nsplit_label = [label[i:i + size] for i in range(0, len(text), size)]\n\ntrain_text_data = split_text[0]\ntest_text_data = split_text[1]\ntrain_label_data = split_label[0]\ntest_label_data = split_label[1]'

In [4]:
# vectorizing
words = []
new_sentences = []

lemmatizer = WordNetLemmatizer()
for sentence in text:
    word = nltk.word_tokenize(sentence.lower())
    lemmatized_output = ' '.join([lemmatizer.lemmatize(w) for w in word])
    words.extend(word)
    new_sentences.append(lemmatized_output)
    
vectorizer = CountVectorizer()
vectorizer.fit(new_sentences)
#print(vectorizer.vocabulary_)
train_text = vectorizer.transform(new_sentences)
train_text = train_text.toarray()

train_label = vectorizer.transform(label)
train_label = train_label.toarray()
#print(vectorizer2.vocabulary_)
#words = sorted(list(set(words)))
print(len(train_text[0]))

82


### Model

In [5]:
# The model
model = Sequential()
model.add(Dense(128, input_shape=(len(train_text[0]),), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(train_label[0]), activation='softmax'))

opt = tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss='sparse_categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

#print(model.summary())
train_label = np.argmax(train_label, axis=1)
model.fit(train_text, train_label, epochs=100, batch_size=32, verbose=1)

#score, acc = model.evaluate(test_text, test_label, batch_size=32)
#print(acc)

Train on 45 samples
Epoch 1/100
45/45 [==============================] - 0s 1ms/sample - loss: 4.3852 - acc: 0.1556
Epoch 2/100
45/45 [==============================] - 0s 68us/sample - loss: 3.5261 - acc: 0.7778
Epoch 3/100
45/45 [==============================] - 0s 77us/sample - loss: 2.2355 - acc: 0.7778
Epoch 4/100
45/45 [==============================] - 0s 73us/sample - loss: 0.9877 - acc: 0.7778
Epoch 5/100
45/45 [==============================] - 0s 74us/sample - loss: 0.7807 - acc: 0.7778
Epoch 6/100
45/45 [==============================] - 0s 78us/sample - loss: 0.6781 - acc: 0.7778
Epoch 7/100
45/45 [==============================] - 0s 71us/sample - loss: 0.5063 - acc: 0.8000
Epoch 8/100
45/45 [==============================] - 0s 73us/sample - loss: 0.3235 - acc: 0.8000
Epoch 9/100
45/45 [==============================] - 0s 81us/sample - loss: 0.3077 - acc: 0.8667
Epoch 10/100
45/45 [==============================] - 0s 79us/sample - loss: 0.2736 - acc: 0.9556
Epoch 11/1

2023-06-02 10:27:13.978107: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 18/100
45/45 [==============================] - 0s 93us/sample - loss: 0.0394 - acc: 1.0000
Epoch 19/100
45/45 [==============================] - 0s 93us/sample - loss: 0.0248 - acc: 1.0000
Epoch 20/100
45/45 [==============================] - 0s 88us/sample - loss: 0.0158 - acc: 1.0000
Epoch 21/100
45/45 [==============================] - 0s 90us/sample - loss: 0.0062 - acc: 1.0000
Epoch 22/100
45/45 [==============================] - 0s 98us/sample - loss: 0.0093 - acc: 1.0000
Epoch 23/100
45/45 [==============================] - 0s 96us/sample - loss: 0.0072 - acc: 1.0000
Epoch 24/100
45/45 [==============================] - 0s 102us/sample - loss: 0.0053 - acc: 1.0000
Epoch 25/100
45/45 [==============================] - 0s 84us/sample - loss: 0.0014 - acc: 1.0000
Epoch 26/100
45/45 [==============================] - 0s 105us/sample - loss: 0.0043 - acc: 1.0000
Epoch 27/100
45/45 [==============================] - 0s 100us/sample - loss: 0.0030 - acc: 1.0000
Epoch 28/100
45/4

45/45 [==============================] - 0s 82us/sample - loss: 6.5152e-04 - acc: 1.0000
Epoch 99/100
45/45 [==============================] - 0s 85us/sample - loss: 4.4682e-05 - acc: 1.0000
Epoch 100/100
45/45 [==============================] - 0s 89us/sample - loss: 5.4066e-05 - acc: 1.0000


# Dialog management

### Preprocessing the user input

In [ ]:
def process_tool(sentence):
    tokens = nltk.word_tokenize(sentence.lower())
    words = [' '.join([lemmatizer.lemmatize(w) for w in tokens])]
    sentence_vect = vectorizer.transform(words)
    sentence_vect = sentence_vect.toarray()
    return sentence_vect

      #get the label for a given text
    
    index = labels[ind]
    return index

def response(intent, data):
    if intent.lower() == "bye":
        print("See you next time.")
    else:
        for info in data["intents"]:
            if info['tag'] == intent[0]:
                print("Chatty:", random.choice(info['responses']))
            break
    return reply

def response():
    while True:
        print("Me:", end="")
        msg = input()
        if msg.lower() == "bye":
            print("Chatty:", "See you next time.")
            break
            
        prediction = model.predict([process_tool(msg)])[0]
        ind = np.argmax(prediction)
        index = label[ind]
            
        for info in file["intents"]:
            if info['tag'] == index:
                print("Chatty:", np.random.choice(info['responses']))

print("You can chat with the bot now!")
print("(Enter 'bye to stop')")

response()

You can chat with the bot now!
(Enter 'bye to stop')
Me:hi
Chatty: Hello, thanks for asking
Me:How you could help me?
Chatty: Hello, thanks for asking
Me: